# Basic usage of the cosmoDC2 catalog and photoz add-on catalog - Weak gravitational lensing shear by dark matter halo

Author : Constantin Payerne

**This notebook can be run at NERSC or CC-IN2P3, where the DESC DC2 products are stored. You need to be a DESC member to be able to access those.**

This notebook shows how to use the cosmoDC2 catalog and the photometric redshift (photo-z) add-on catalog to estimate the average excess surface density around galaxy clusters (i.e. to produce the data vector needed for WL mass estimation).

After going through this notebook, you should be able to:

- extract a dark matter halo catalog from cosmoDC2,
- load the background galaxy catalog from cosmoDC2 (around a given halo),
- access the photo-z of background galaxies from the photoz add-on catalog,
- compute the excess surface density estimator using the above.

Along with standard packages, you need to have the [CLMM](https://github.com/LSSTDESC/CLMM) library installed to run this notebook.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.integrate
from astropy.table import Table, join
import healpy

In [ ]:
import clmm 
import GCRCatalogs

NB: A detailled GitHub repository with multiple example the use of GCRCatalogs is available at https://github.com/LSSTDESC/DC2-analysis/tree/master/tutorials.

# 1. Identify a cluster (massive dark matter halo) in cosmoDC2

Here, we load the cosmoDC2 exctragalactic catalog (small version)

In [ ]:
extragalactic_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In order to speed up the extraction of cosmoDC2 data, we select a dark matter halo within a specified healpix pixel available in `cosmoDC2_v1.1.4_small`. The user can change the selected healpix pixel by modifying the quantity `index_healpix_pixels_selected` below.

In [ ]:
healpix_pixels = extragalactic_cat.get_catalog_info()['healpix_pixels']
print(healpix_pixels)

In [ ]:
index_healpix_pixels_selected = 6

In [ ]:
healpix_selected = healpix_pixels[index_healpix_pixels_selected]
print('The selected healpix pixel is ' + str(healpix_selected))

We extract the dark matter halo catalog for this specified healpix pixel by
- asking that the cosmoDC2 galaxy is a central galaxy (of a halo), with the flag `is_central=True`. 
- applying some property cut (here, mass and redshift range). 

In [ ]:
# get list of massive halos in a given redshift and mass range

mmin = 1.e14 # Msun
zmin = 0.2
zmax = 0.3
massive_halos = extragalactic_cat.get_quantities(['halo_mass','redshift','ra', 'dec', 'halo_id'],
                                                 filters=[f'halo_mass > {mmin}','is_central==True',
                                                          f'redshift>{zmin}', f'redshift<{zmax}'], native_filters=['healpix_pixel == ' + str(healpix_selected)])
N_halo = len(massive_halos['halo_mass'])

print(f'There are {N_halo} clusters in this mass and redshift range, in this healpix pixel')

We then select the most massive halo in the recovered sample.

In [ ]:
select = massive_halos['halo_mass'] == np.max(massive_halos['halo_mass'])
ra_halo = massive_halos['ra'][select][0]
dec_halo = massive_halos['dec'][select][0]
z_halo = massive_halos['redshift'][select][0]
mass_halo =massive_halos['halo_mass'][select][0]
id_halo = massive_halos['halo_id'][select][0]

In [ ]:
print (f'The most massive cluster has ID {id_halo}')
print(f'the halo is in ra = {ra_halo:.2f} deg, dec = {dec_halo:.2f} deg, with redshift = {z_halo:.2f} and mass = {mass_halo:.2e} Msun')

# 2. Extract the galaxy catalog

The galaxy shapes and true redshifts are available in the cosmoDC2 extragalaxtic catalog, and their photometric redshifts (measured using the BPZ or FlexZboost codes) are available in separate add-on catalog. 
- We first extract the galaxies within an aperture of 0.6 degrees arround the cluster position, and with magnitude cut `mag_i < 25`.
- Then move to the add-on catalog to get the corresponding photoz.

## 2.1 Select galaxies in the cluster field
NB: at this stage, we load all galaxies in the field. Background galaxy selection will come in section 4.

In [ ]:
# Position and magnitude cuts
ra_min, ra_max = ra_halo - 0.3, ra_halo + 0.3
dec_min, dec_max = dec_halo - 0.3, dec_halo + 0.3
mag_i_max = 25

We indentify the contiguous healix pixels corresponding to the window using the `healpy` package.

In [ ]:
n_random = 100000
ra_random = np.random.random(n_random)*(ra_max-ra_min) + ra_min
dec_random = np.random.random(n_random)*(dec_max-dec_min) + dec_min
unique_healpix_in_window = list(np.unique(healpy.ang2pix(32, ra_random, dec_random, nest=False, lonlat=True)))
print('The selected contiguou healpix pixels within the window are ' + str(unique_healpix_in_window))

Extract the needed quantities for each galaxy:
- complex shear components `'shear_1'`, `'shear_2'`
- intrinsic ellipticity components `'ellipticity_1_true'`, `'ellipticity_2_true'`
- convergence `'convergence'`
- `ra`, `dec`, `redshift`

In [ ]:
coord_filters = ['ra >= {}'.format(ra_min),'ra < {}'.format(ra_max),'dec >= {}'.format(dec_min),'dec < {}'.format(dec_max)]
z_filters = ['redshift >= 0']
mag_filters = ['mag_i < {}'.format(mag_i_max)]
extragalactic_cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_image", config_overwrite=dict(healpix_pixels=unique_healpix_in_window))

gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 
                                            'ra', 'dec',
                                            'shear_1', 'shear_2', 
                                            'ellipticity_1_true', 'ellipticity_2_true',
                                            'redshift', 'convergence', 'halo_id'],
                                            filters=(coord_filters + z_filters + mag_filters), 
                                          )

print(str(len(gal_cat['galaxy_id'])) +' loaded galaxies')

We then compute the obseved ellipticities using the shear, the convergence and the intrinsic ellipticity for each galaxy, using [CLMM](https://github.com/LSSTDESC/CLMM). The lensed ellipticity writes for a single galaxy

$$
\epsilon^{\rm obs} =
        \frac{\epsilon^{\rm int} + g}{1 + g^*\epsilon^{\rm int}},$$ where the reduced shear expresses as $$g = \frac{\gamma}{1-\kappa}.$$

In [ ]:
gal_cat['e1'] = clmm.utils.compute_lensed_ellipticity(gal_cat['ellipticity_1_true'], 
                                                      gal_cat['ellipticity_2_true'], 
                                                      gal_cat['shear_1'], gal_cat['shear_2'], 
                                                      gal_cat['convergence'])[0]

gal_cat['e2'] = clmm.utils.compute_lensed_ellipticity(gal_cat['ellipticity_1_true'], 
                                                      gal_cat['ellipticity_2_true'], 
                                                      gal_cat['shear_1'], 
                                                      gal_cat['shear_2'], 
                                                      gal_cat['convergence'])[1]

In [ ]:
# rename the redshift column
gal_cat['z'] = gal_cat['redshift']

In [ ]:
# change gal_cat to an Astropy Table for later use with CLMM
gal_cat = Table(gal_cat)

## 2.2 Member galaxy identification for the selected cluster

From the galaxy catalog `gal_cat` (that contains all galaxies in the field, independent of redshift), the galaxies belonging to the dark matter halo (cluster) can be identified from the `halo_id` field.

In [ ]:
member_gal_catalog = gal_cat[gal_cat['halo_id'] == id_halo]
print(f"There are {len(member_gal_catalog)} member galaxies in that halo")

# 3. Extract photoz information

The full details of the photo-z add-on catalog `'cosmoDC2_v1.1.4_small_with_photozs_v1'` are given in the [photoz catalog notebook](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/extragalactic_gcr_photoz_catalogs.ipynb). This catalog contains photometric redshifts that were computed with the [BPZ template-based code](https://ui.adsabs.harvard.edu/abs/2000ApJ...536..571B/abstract) and is the one we use. Note that the catalog `'cosmoDC2_v1.1.4_small_with_photozs_flexzboost_v1'` contains photozs from [FleXZBoost machine learning code](https://arxiv.org/abs/2001.03621) and could be used instead.

## 3.1 Load photoz quantities for each galaxy
The quantities of interest are
- the probability density function `'photoz_pdf'`,
- the mean photometric redshift `'photoz_mean'`.

NB: The add-on catalog does not support geometrical cut (in `ra`, `dec`, `redshift`). We first extract the photo-z informations in the corresponding healpix pixel, we then match with the cosmoDC2 galaxy sample.



In [ ]:
# Load the photoz catalog
extragalactic_photoz_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small_with_photozs_v1')
z_bins = extragalactic_photoz_cat.photoz_pdf_bin_centers

In [ ]:
# Get the redshift information
photoz_cat = extragalactic_photoz_cat.get_quantities(['photoz_pdf','photoz_mean','redshift', 'galaxy_id', 'photoz_odds'], 
                                                     native_filters=[(lambda p: np.isin(p, [unique_healpix_in_window]), "healpix_pixel")])

In [ ]:
# Convert to astropy table
photoz_cat_astropy = Table()
photoz_cat_astropy['galaxy_id'] = photoz_cat['galaxy_id']
photoz_cat_astropy['photoz_pdf'] =  photoz_cat['photoz_pdf']
photoz_cat_astropy['photoz_mean'] =  photoz_cat['photoz_mean']
photoz_cat_astropy['photoz_odds'] =  photoz_cat['photoz_odds']

In [ ]:
# Match the galaxy catalog and photoz catalog (using `galaxy_id`) and store in a new astropy table `cat_complete`
cat_complete = join(photoz_cat_astropy, gal_cat, keys = 'galaxy_id')

## 3.2 Add derived photoz information to the table

For each galaxy, we calculate the probability of being in the background of the cluster, i.e. of having a redshift $>z_{\rm halo} + \delta$, where delta is a positive value (in this notebook, we choose $\delta = 0.$, which can be different for other analysis, depending on uncertaintiy on cluster redshift).

$$p_i = \int_{z_{\rm halo} + \delta}^{+\infty} dz\ p_{\rm photoz, i}(z)$$ where $p_{\rm photoz, i}$ is the photometric redshift probability density function for the galaxy $i$.

In [ ]:
delta = 0.
z_min = z_halo + delta

In [ ]:
axis_select_background = [0 if z_bins[i] < z_min else 1 for i in range(len(z_bins))]

In [ ]:
m = np.zeros([len(cat_complete), len(z_bins)])
for i in range(len(cat_complete)):
    m[i,:] = np.array(cat_complete['photoz_pdf'][i])

In [ ]:
integral_pdf_behind = scipy.integrate.simps(m * axis_select_background, x = z_bins, axis = 1)
norm_pdf = scipy.integrate.simps(m, x = z_bins, axis = 1)

In [ ]:
# add the probability to be a background galaxy and the normalisation of the photoz pdf to the table.
cat_complete['background_probability'] = integral_pdf_behind / norm_pdf
cat_complete['photoz_pdf_norm'] = norm_pdf

# 4. Background source selection and contamination estimation

We discuss briefly the effect of redhsift selection of background galaxies, that is affected by the contamination of 'fake' background galaxies i.e. galaxies with true redshift below the halo redshift, but identified as background redshift due to specific cut on photometric information. In the same way, member galaxies can be identified as background galaxies. To test 2 different selection cuts, we consider 3 different cases:
- `cat_true`: cut on true galaxy redshift $z_{\rm cosmoDC2} > z_{\rm l} + \delta$ (identifying true background galaxies).
- `cat_photoz`: cut on mean photoz galaxy redshift $\langle z \rangle_{\rm photoz} > z_{\rm l} + \delta$,
- `cat_probability`: cut on background probability $\int_{z_{\rm halo} + \delta}^{+\infty} dz\ p_{\rm photoz, i}(z)  > 0.9$.

In [ ]:
cat_true = cat_complete[cat_complete['redshift'] > z_min]
cat_photoz = cat_complete[cat_complete['photoz_mean'] > z_min]
cat_probability = cat_complete[cat_complete['background_probability'] > 0.9]

We can plot the background probability as a function of true redshift. We see that the majority of 'true' background galaxies are well labelled as observed background galaxies. However, we see that few galaxies have non-zero probability to be in the foreground of the cluster.

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (7,5))
plt.hist2d(cat_complete['redshift'] ,cat_complete['background_probability'], cmin = 1, bins = [np.linspace(0,3,30), np.linspace(0,1,30)], cmap = 'PuRd')
plt.hist(cat_complete['redshift'], density = True, histtype = 'step', bins = 30, label = r'$n_{\rm gal}(z_{\rm true})$')
plt.colorbar()
plt.vlines(z_halo, 0, 1, color = 'k', linestyle = '--', zorder = 1000, linewidth = 3)
plt.ylabel(r'$P(z_s > z_l)$', fontsize = 25)
plt.xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 25)
plt.tick_params(axis='both', which = 'major', labelsize= 15, zorder = 0)
plt.legend(fontsize = 20, loc = 'lower right', frameon = False, framealpha = 1)
plt.ylim(0.05,1.05)
plt.xlim(0,3)

We check the contamination of member galaxies by counting the member galaxies and also the foreground galaxies in each dataset:

In [ ]:
print('====cat_true====')
member_gal_true = len(cat_true[np.isin(cat_true['galaxy_id'],member_gal_catalog['galaxy_id'])])
fraction_member_true = 100*member_gal_true/len(cat_true)
print(f'member : {member_gal_true} ({fraction_member_true:.2f} %)')
foreground_gal_true = len(cat_true['redshift'][cat_true['redshift'] < z_halo])
fraction_foreground_true = 100*foreground_gal_true/len(cat_true)
print(f'foreground : {foreground_gal_true} (fraction {fraction_foreground_true:.2f} %)')

print('\n====cat_photoz====')
member_gal_photoz = len(cat_photoz[np.isin(cat_photoz['galaxy_id'],member_gal_catalog['galaxy_id'])])
fraction_member_photoz = 100*member_gal_photoz/len(cat_photoz)
print(f'member : {member_gal_photoz} ({fraction_member_photoz:.2f} %)')
foreground_gal_photoz = len(cat_photoz['redshift'][cat_photoz['redshift'] < z_halo])
fraction_foreground_photoz = 100*foreground_gal_photoz/len(cat_photoz)
print(f'foreground : {foreground_gal_photoz} (fraction {fraction_foreground_photoz:.2f} %)')

print('\n====cat_probability====')
member_gal_probability = len(cat_probability[np.isin(cat_probability['galaxy_id'],member_gal_catalog['galaxy_id'])])
fraction_member_probability = 100*member_gal_probability/len(cat_probability)
print(f'member : {member_gal_probability} ({fraction_member_probability:.2f} %)')
foreground_gal_probability = len(cat_probability['redshift'][cat_probability['redshift'] < z_halo])
fraction_foreground_probability = 100*foreground_gal_probability/len(cat_probability)
print(f'foreground : {foreground_gal_probability} (fraction {fraction_foreground_probability:.2f} %)')

**From the count of contamination in each data set, we can see that the background galaxy selection based on mean photometric redshift induces a larger fraction of true foreground galaxies and halo member galaxies in the galaxy sample.** 

As a recap, we show in the figure below the different true redshifts histograms of the datasets obtained with the different redshift cuts:
- the true redshift histogram in the selected healpix (orange)
- the true redshift histogram in the `cat_photoz` dataset (blue)
- the true redshift histogram in the `cat_photoz` dataset and applying on the quality criteria `photoz_odds` > 0.8, where `photoz_odds` is the integral of the photoz pdf over a fixed redshift range arround the maximum of the pdf, see [photoz catalog notebook](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/extragalactic_gcr_photoz_catalogs.ipynb) for more details (shaded blue)
- the true redshift histogram in the `cat_probability` dataset (red dashed)
- the average photoz-pdf of each member galaxies (cyan)

In [ ]:
# catalog of member galaxies containing all photoz information
member_gal_photoz_catalog = cat_complete[np.isin(cat_complete['galaxy_id'],member_gal_catalog['galaxy_id'])]

In [ ]:
pdf_member_galaxy_average = 0
for i, pdf in enumerate(member_gal_photoz_catalog['photoz_pdf']):
    pdf_member_galaxy_average = pdf_member_galaxy_average + pdf/member_gal_photoz_catalog['photoz_pdf_norm'][i]

In [ ]:
plt.figure(figsize = (7,5))
plt.rcParams['axes.linewidth'] = 2
bins = np.linspace(0, 3, 56)
plt.tick_params(axis='both', which = 'both', labelsize= 15)
hist = plt.hist(gal_cat['redshift'], bins = bins, linewidth = 0, label = r'$z_{\rm cosmoDC2}$ distribution',  color = 'orange', alpha = 0.5)
plt.hist(cat_photoz['redshift'], bins = bins, histtype = 'step', linewidth = 2, label = r'cut $: \langle z \rangle_{\rm photoz} > z_{\rm halo}$', color = 'b');
mask_odds = cat_photoz['photoz_odds'] > 0.8
plt.hist(cat_photoz['redshift'][mask_odds], bins = bins, alpha = 0.1, linewidth =0, label = r'cut $: \langle z \rangle_{\rm photoz} > z_{\rm halo}$ , odds > 0.8', color = 'b', linestyle = '-.');
plt.hist(cat_probability['redshift'], bins = bins, linewidth = 2, histtype = 'step', label = r'cut $: P(z > z_{\rm halo})  > 0.9$',color = 'r', linestyle = '--');
plt.axvline(z_halo, color = 'k', linestyle = '-', label = 'halo redshift', zorder = 1000)
pdf = pdf_member_galaxy_average/len(member_gal_photoz_catalog)
plt.plot(z_bins, max(hist[0].flatten())*pdf/max(pdf), 'cyan', label = 'average member galaxy photoz pdf', linewidth = 3)
plt.ylim(0, max(hist[0].flatten())+ 100)
plt.legend(frameon = False, fontsize = 12)
plt.xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 30)
plt.ylabel(r'Histogram', fontsize = 30)
plt.title('Selection effect on background galaxies', fontsize = 20)
plt.show()

We note that the combination of the redshift cut based on the mean photometric redshift and the odds cut has reduced the proportion of foreground galaxies, but has significantly reduced the size of the sample. 

# 5. Compute the excess surface density profile of the cluster

### Estimator of the excess surface density

- The maximum likelihood estimator of the excess surface density in the radial bin $[R, R + \Delta R[$ is defined by

$$\widehat{\Delta\Sigma}_+(R) = \frac{1}{\sum\limits_{s = 1} w_{s}}
     \sum\limits_{s= 1}w_{s}\langle\Sigma_{{\rm crit}}(z_s, z_l)^{-1}\rangle^{-1}\epsilon_+^{s},$$ where $\epsilon_+^{s}$ is the tangential ellipticity of the background galaxy with index $s$ (as source) relative to the dark matter halo position with index $l$ (as lens).
     
- The critical surface mass density expresses as $\Sigma_{{\rm crit}}(z_s, z_l) = \frac{c^2}{4 \pi G} \frac{D_A(z_s)}{D_A(z_l) D_A(z_s, z_l)}$, where $D_A(z_l), D_A(z_s)$ are respectively the angular diameter distance to the lens and to the source in physical units, and $D_A(z_s, z_l)$ is the angular diameter distance between the lens and the source.


- The average $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle$ is defined as $\langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle = \int_{z_l + \delta}^{+\infty} d z_s\ p_{\rm photoz,s}(z_s)\ \Sigma_{\rm crit}(z_s,z_l)^{-1}$, where $p_{\rm photoz,s}$ is the photometric probability density function for the background galaxy with index $s$.

### Weights $w_s$

The quantities $w_{s}$ are the weights that maximise the sigmnal-to-noise ratio of the excess surface density estimator. They downweight the galaxies that are close in redshift to the cluster (where the lensing signal is weak). They include the lack of informations on both redshift and shape reconstruction for each background galaxies. 

- In the case where there is no error on the shape measurement (for the purpose of cosmoDC2 galaxies), the weight writes $w_{s} = \langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle^{2}$,

- In the case of true redshift $z_s$, the PDF reduces to a Dirac function centered at $z_s$, giving the average $\langle\Sigma_{\rm crit}(z_l,z_s)^{-1}\rangle = \Sigma_{\rm crit}(z_l,z_s)^{-1} $ and the weight $w_{s} = \Sigma_{\rm crit}(z_s,z_l)^{-2}$,


In [ ]:
# Define a CLMM cosmology object; used to compute the critical surface density
cosmo = clmm.Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

In [ ]:
# Inverse critical surface density. Recall that z_bins are the bin centers of the photoz pdf.
sigma_crit_1 = np.array(1/cosmo.eval_sigma_crit(z_halo, z_bins))

In [ ]:
m = np.zeros([len(cat_complete), len(z_bins)])
for i in range(len(cat_complete)):
    m[i,:] = np.array(cat_complete['photoz_pdf'][i])

In [ ]:
# compute the average inverse critical surface density
unormed_integral = scipy.integrate.simps(m * sigma_crit_1, x = z_bins, axis = 1)
norm = scipy.integrate.simps(m, x = z_bins, axis = 1)

**We compute the excess surface density in 3 different cases:**

- using true backgroud galaxy redshift : $w_{ls}^{\rm true} = \Sigma_{\rm crit}(z_{\rm cosmoDC2},z_l)^{-2}$
- using photometric probability density function : $w_{ls}^{\rm pdf} = \langle\Sigma_{\rm crit}(z_s,z_l)^{-1}\rangle^{2}$
- using mean photometric redshift $w_{ls}^{\rm mean} = \Sigma_{\rm crit}(\langle z \rangle_{\rm photoz},z_l)^{-2}$ 
- using random sample redshift from the photoz pdf : $w_{ls}^{\rm mean} = \Sigma_{\rm crit}( z_{\rm random},z_l)^{-2}$

In [ ]:
# define the various weights, depending of the case considered
ws_pdf = (unormed_integral/norm)**2
ws_true = ( 1. /cosmo.eval_sigma_crit(z_halo, cat_complete['redshift']) )**2 
ws_mean = ( 1. /cosmo.eval_sigma_crit(z_halo, cat_complete['photoz_mean']) )**2 

In [ ]:
ws_random = []
for pdf in cat_complete['photoz_pdf']:
    ws_random.append(np.random.choice(1./cosmo.eval_sigma_crit(z_halo, z_bins), p = pdf/np.sum(pdf))**2) 
ws_random = np.array(ws_random)

#### We plot the probability $p_{s} = \frac{w_{s}}{\sum_{s'} w_{s'}}$ in each case.

In [ ]:
ws_with_photozs = [ws_pdf, ws_mean, ws_random]
label = [r'$p_{s}^{\rm pdf}$', 
         r'$p_{s}^{\rm mean}$', 
         r'$p_{s}^{\rm random}$']
title = []

x = np.linspace(0, 7, 100)
fig, axs = plt.subplots(1, 3, figsize = (25,6))
for i in range(3):
    axs[i].hist2d(cat_complete['redshift'], ws_with_photozs[i]/np.sum(ws_with_photozs[i]), cmin = 1, bins = [np.linspace(0,3,30), np.linspace(0,5e-5,30)], cmap = 'PuRd', label = label[i])
    axs[i].scatter(cat_complete['redshift'], ws_true/np.sum(ws_true), s = 1, c = 'k', label = r'$p_{s}^{\rm true}$')
    axs[i].vlines(z_halo, 0, 1, color = 'magenta', linestyle = '--', zorder = 1000, linewidth = 3)
    axs[i].set_ylabel(label[i], fontsize = 25)
    axs[i].set_xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 25)
    axs[i].tick_params(axis='both', which = 'major', labelsize= 15, zorder = 0)
    axs[i].legend(fontsize = 20, loc = 'lower right', frameon = True, framealpha = 1)
    axs[i].set_ylim(0,max(ws_with_photozs[0]/np.sum(ws_with_photozs[0])))

For the thee cases using photoz, the weights $p_s$ scatter arround the black curves below $z \approx 1.5$, beyond this redshift, the occurences of catastrophic photometric redshift reconstruction increases and the deviation to the black lines is larger. Additive quality cut cat be applied to remove catastrophic redshift reconstruction from the data set

In [ ]:
#plt.figure(figsize = (7,5))
#mask = cat_complete['background_probability'] > 0.9
#plt.hist2d(cat_complete['redshift'][mask], ws_pdf[mask]/np.sum(ws_pdf[mask]), cmin = 1, bins = [np.linspace(0,3,30), np.linspace(0,5e-5,30)], cmap = 'PuRd', label = r'$w_s^{photoz}$')
#plt.colorbar()
#plt.scatter(cat_complete['redshift'], ws_true/np.sum(ws_true), s = 3, c = 'k')
#plt.plot([],[],'k' , label = r'$w_s^{true}$')
#plt.vlines(z_halo, 0, 1, color = 'magenta', linestyle = '--', zorder = 1000, linewidth = 3)
#plt.ylabel(r'$w_s$', fontsize = 25)
#plt.xlabel(r'$z_{\rm cosmoDC2}$', fontsize = 25)
#plt.tick_params(axis='both', which = 'major', labelsize= 15, zorder = 0)
#plt.legend(fontsize = 20, loc = 'lower right', frameon = True, framealpha = 1)
#plt.ylim(0,max(ws_with_photozs[0]/np.sum(ws_with_photozs[0])))

In [ ]:
cat_complete['ws_true'] = np.array(ws_true)
cat_complete['ws_pdf'] = np.array(ws_pdf)
cat_complete['ws_mean'] = np.array(ws_mean)
cat_complete['ws_random'] = np.array(ws_random)

In [ ]:
data_complete = clmm.GCData(cat_complete)
cl_complete = clmm.GalaxyCluster('DM_halo', ra_halo, dec_halo, z_halo, data_complete)

We compute the tangential ellipticity for each background galaxy relative to the dark matter halo center (we use the `CLMM` package)

In [ ]:
cl_complete.compute_tangential_and_cross_components(
                                    shape_component1='e1', shape_component2='e2', 
                                    tan_component='shear_t', cross_component='shear_x',
                                    geometry="flat")

# add projected radial distance of the galaxy to the cluster center to the table
cl_complete.galcat['r'] = cosmo.eval_da(cl_complete.z)*cl_complete.galcat['theta']

We create a galaxy catalog `cl_true` with background selection on the true redshift. This is the ideal situation where there is no mis-identification of background gaalxies.

In [ ]:
data_true = clmm.GCData(cl_complete.galcat[cl_complete.galcat['redshift'] > z_min])
cl_true = clmm.GalaxyCluster('DM_halo', ra_halo, dec_halo, z_halo, data_true)

#### To compute the excess surface density,

- We define a set of bin edges,

- We compute the average excess surface density in each bins as described above,

- We compute the errorbars as the standard deviation in each bin.

In [ ]:
bin_edges = clmm.make_bins(.5,  5, nbins=10, method='evenlog10width')
radial_bins = [[bin_edges[i], bin_edges[i + 1]] for i in range(len(bin_edges)-1)]

In [ ]:
def make_radial_profile(catalog, radial_bins, radius='r', tangential_component='shear_t', weight = 'ws_true'):
    '''
    Evaluates the contamination from member galaxies and foreground galaxies
    in each radial bin of the profile
    
    Parameters:
    -----------
        catalog: CLMM galaxy cluster object
            Contains the galcat table
        radial_bin: array-like
            Radial binning of the profile
        radius: str
            Name of the radius column in the galcat table
        tangential_component: str
            Name of the tangential shear component column in the galcat table
        weights: str
            Name of the weights column in the galcat table
                
    Returns:
    --------
        profile: astropy table
            Excess surface density profile and corresponding errors
    ''' 

    profile = Table(names=['r', 'DS', 'DS_err'])
    
    for i, r_bin in enumerate(radial_bins):     
        mask = (catalog.galcat[radius] > r_bin[0])*(catalog.galcat[radius] <= r_bin[1])*(catalog.galcat[weight] != 0)
        data_cut = catalog.galcat[mask]
        
        if len(data_cut) == 0:          
            profile.add_row([r_mean, None, None])
            continue
        
        r_mean = np.mean(data_cut['r'])
        ds = np.average(data_cut[tangential_component]*data_cut[weight]**(-0.5), 
                             weights = data_cut[weight])
        
        # Standard deviation of the excess surface density        
        ds_rms = np.sqrt(np.average((data_cut[tangential_component]*data_cut[weight]**(-0.5) - ds)**2, 
                                         weights = data_cut[weight]))/np.sqrt(len(data_cut))

        profile.add_row([r_mean, ds, ds_rms])
        
    return profile

## Contamination effect

We create two CLMM galaxy cluster objects, based on the 2 different background galaxy selection criteria:
- `cl_photoz_mean` with cut :  $\langle z \rangle > z_{\rm halo} + \delta$
- `cl_photoz_prob` with cut : $P(z > z_{\rm halo} + \delta)  > 0.9$

In [ ]:
data_photoz_mean = clmm.GCData(cl_complete.galcat[cl_complete.galcat['photoz_mean'] > z_min])
cl_photoz_mean = clmm.GalaxyCluster('DM_halo', ra_halo, dec_halo, z_halo, data_photoz_mean)

In [ ]:
data_photoz_mean_odds = clmm.GCData(cl_complete.galcat[(cl_complete.galcat['photoz_mean'] > z_min)*(cl_complete.galcat['photoz_odds'] > .95)])
cl_photoz_mean_odds = clmm.GalaxyCluster('DM_halo', ra_halo, dec_halo, z_halo, data_photoz_mean)

In [ ]:
data_photoz_prob = clmm.GCData(cl_complete.galcat[cl_complete.galcat['background_probability'] > 0.9])
cl_photoz_prob = clmm.GalaxyCluster('DM_halo', ra_halo, dec_halo, z_halo, data_photoz_prob)

In [ ]:
def contamination_profile(catalog, radial_bin, radius='r'):
    '''
    Evaluates the contamination from member galaxies and foreground galaxies
    in each radial bin of the profile
    
    Parameters:
    -----------
        catalog: CLMM galaxy cluster object
            Contains the galcat table
        radius: str
            Name of the radius column in the galcat table
        radial_bin: array-like
            Radial binning of the profile
            
    Returns:
    --------
        profile: astropy table
            Contamination profile
    ''' 

    profile = Table(names=['r', 'count_foreground', 'count_member'])
    
    for i, r_bin in enumerate(radial_bin):       
        mask_foreground = (catalog.galcat['redshift'] < z_halo)
        mask_radius = (catalog.galcat[radius] > r_bin[0])*(catalog.galcat[radius] <= r_bin[1])
        data_cut = catalog.galcat[mask_radius]
        if len(data_cut) == 0:            
            continue
        r_mean = np.mean(data_cut['r'])      
        data_cut_foreground = catalog.galcat[mask_radius*mask_foreground]
        data_member = catalog.galcat['galaxy_id'][mask_radius][np.isin(catalog.galcat['galaxy_id'][mask_radius],member_gal_catalog['galaxy_id'])]
        profile.add_row([r_mean, len(data_cut_foreground)/len(data_cut), len(data_member)/len(data_cut)])
    return profile

We check the contamination of member galaxies and foreground galaxies in each dataset

In [ ]:
bin_centers = np.mean(radial_bins, axis = 1)

contamination_photoz_mean = contamination_profile(cl_photoz_mean, radial_bins, radius = 'r')
contamination_photoz_mean_odds = contamination_profile(cl_photoz_mean_odds, radial_bins, radius = 'r')
contamination_photoz_prob = contamination_profile(cl_photoz_prob, radial_bins, radius = 'r')

In [ ]:
plt.figure(figsize = (7,5))
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
#plt.plot(contamination_photoz_mean['r'], 100*contamination_photoz_mean['count_foreground'],\
#         'b', label = 'photoz mean : foreground galaxies', linewidth = 3)
#plt.plot(contamination_photoz_mean['r'], 100*contamination_photoz_mean['count_member'],\
#         '-b', label = r'$\langle z \rangle_{\rm photoz} > z_{\rm halo}$', linewidth = 3)
plt.plot(contamination_photoz_mean['r'], 100*contamination_photoz_mean_odds['count_member'],\
         '-b', label = r'$\langle z \rangle_{\rm photoz} > z_{\rm halo}$', linewidth = 3)
#plt.plot(contamination_photoz_prob['r'], 100*contamination_photoz_prob['count_foreground'],'r', \
#         label = 'photoz prob. : foreground galaxies', linewidth = 3)
plt.plot(contamination_photoz_prob['r'], 100*contamination_photoz_prob['count_member'],'-r', \
         label = r'$P(z_{\rm gal} > z_{\rm halo})  > 0.9$', linewidth = 3)
#plt.xlim(min(bin_centers),max(bin_centers))
plt.xlabel('R [Mpc]', fontsize = 20)
plt.xscale('log')
plt.grid(True, which='both')
plt.ylabel(f'Contamination (%)', fontsize = 20)
plt.legend(frameon = True, framealpha = 1, fontsize = 17, loc = 'best')

In [ ]:
plt.figure(figsize = (7,5))
plt.grid(True, which='both')
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
plt.plot(contamination_photoz_mean['r'], 100*contamination_photoz_mean['count_foreground'],\
         'b', label = 'photoz mean : foreground galaxies', linewidth = 3)
plt.plot(contamination_photoz_prob['r'], 100*contamination_photoz_prob['count_foreground'],'r', \
         label = 'photoz prob. : foreground galaxies', linewidth = 3)
plt.xlim(min(bin_centers),max(bin_centers))
plt.xlabel('R [Mpc]', fontsize = 20)
plt.ylabel(f'Contamination (%)', fontsize = 15)
plt.legend(frameon = False, fontsize = 17, loc = 'best')

As mentionned in section 4, the background galaxy selection is more efficient when selecting galaxies on their probabilities to be in the background of the halo. As expected, the contamination from member galaxies decreases with the radius, since the contamination of foreground galaxies is roughly constant with R.

We now plot the excess surface density profile for the 3 galaxy catalogs, using the weight $w^{\rm pdf}_{ls}$ for each galaxy.

In [ ]:
weight_name = 'ws_pdf'

In [ ]:
#true redhift
profile_true = make_radial_profile(cl_true, radial_bins, radius = 'r', 
                                   tangential_component = 'shear_t', 
                                   weight = weight_name
                                   )

profile_photoz = make_radial_profile(cl_photoz_mean, radial_bins, radius = 'r', 
                                  tangential_component = 'shear_t', 
                                  weight = weight_name
                                  )

profile_probability = make_radial_profile(cl_photoz_prob, radial_bins, radius = 'r', 
                                  tangential_component = 'shear_t', 
                                  weight = weight_name
                                  )

We use the CLMM DESC code to compute the 'fiducial' excess surface mass density, by using a NFW profile with halo mass of 1e14 M_sun. We choose a concentration of 5.

In [ ]:
moo = clmm.Modeling(massdef='mean', delta_mdef=200, halo_profile_model='nfw')
moo.set_cosmo(cosmo)
moo.set_concentration(5)
moo.set_mass(1e14)
r = np.logspace(np.log10(0.5), np.log10(6), 100)
DeltaSigma = moo.eval_excess_surface_density(r, z_halo)

In [ ]:
ylabel = r'$\Delta\Sigma$ ' +'$[$' + r'${\rm M}$' + r'$_\odot\;$'+ r'${\rm Mpc}$'+r'$^{-2}$'r'$]$'
xlabel = r'$R\ [$' + r'${\rm Mpc}$' + r'$]$'
plt.figure(figsize = (10,7))
plt.grid(True, which="both", ls="-")
plt.rcParams['axes.linewidth'] = 2
plt.tick_params(axis='both', which = 'both', labelsize= 15)
deltalogr = 0.03
dr = deltalogr*profile_true['r']

plt.errorbar(profile_true['r']   + -1*dr, profile_true['DS']   , profile_true['DS_err'], label = r'true - z',
            capsize = 8,
             fmt = ' ', marker = '^', 
             markersize = 10, 
             markerfacecolor = 'w',
            )
plt.errorbar(profile_probability['r']   +1*dr, profile_probability['DS']   , profile_probability['DS_err'], label = r'BPZ photoz',
             capsize = 8,
             fmt = ' ', marker = 'o', markersize = 10, markerfacecolor = 'w'
            
            )
plt.plot(r, DeltaSigma,'-k', label = 'NFW 1-halo term')
plt.xscale('log')
plt.xlim(min(r),max(r))
#plt.ylim(-0.2e14,0.8e14)
plt.xlabel(xlabel, fontsize = 20)
plt.ylabel(ylabel, fontsize = 20)
plt.legend(frameon = True, fontsize = 20, loc = 'upper right', framealpha = 1)
plt.title('Excess surface density', fontsize = 20)
plt.show()